# <center>[Histopathologic Cancer Detection](https://www.kaggle.com/competitions/histopathologic-cancer-detection/overview)</center>
---

## [About the data](https://www.kaggle.com/competitions/histopathologic-cancer-detection/data)
In this dataset, you are provided with a large number of small pathology images to classify. Files are named with an image **`id`**. The **`train_labels.csv`** file provides the ground truth for the images in the **`train`** folder. You are predicting the labels for the images in the **`test`** folder. A **`positive label`** indicates that the center **`32x32px`** region of a patch contains at least one pixel of tumor tissue. Tumor tissue in the outer region of the patch does not influence the label. This outer region is provided to enable fully-convolutional models that do not use zero-padding, to ensure consistent behavior when applied to a whole-slide image.

The original PCam dataset contains duplicate images due to its probabilistic sampling, however, the version presented on Kaggle does not contain duplicates. We have otherwise maintained the same data and splits as the PCam benchmark.

## Importing the packages

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch

from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchmetrics.functional import accuracy
import pytorch_lightning as pl

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
print("pandas version:",pd.__version__)
print("numpy version:",np.__version__)
#print("seaborn version:",sns.__version__)
print("torch version:",torch.__version__)
print("pytorch ligthening version:",pl.__version__)

pandas version: 1.5.3
numpy version: 1.23.5
torch version: 2.0.0+cpu
pytorch ligthening version: 2.0.4


## Collecting the data

In [3]:
! tree -d /kaggle

/kaggle
├── input
│   └── histopathologic-cancer-detection
│       ├── test
│       └── train
├── lib
│   └── kaggle
├── src
└── working

8 directories


In [4]:
! head /kaggle/input/histopathologic-cancer-detection/train_labels.csv

id,label
f38a6374c348f90b587e046aac6079959adf3835,0
c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
755db6279dae599ebb4d39a9123cce439965282d,0
bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
068aba587a4950175d04c680d38943fd488d6a9d,0
acfe80838488fae3c89bd21ade75be5c34e66be7,0
a24ce148f6ffa7ef8eefb4efb12ebffe8dd700da,1
7f6ccae485af121e0b6ee733022e226ee6b0c65f,1
559e55a64c9ba828f700e948f6886f4cea919261,0


In [5]:
data_df = pd.read_csv(filepath_or_buffer="/kaggle/input/histopathologic-cancer-detection/train_labels.csv")
data_df.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [6]:
# Shape of the data
data_df.shape

(220025, 2)

In [7]:
# Target feature value_counts
print(data_df["label"].value_counts())
data_df["label"].value_counts(normalize=True) * 100

0    130908
1     89117
Name: label, dtype: int64


0    59.496875
1    40.503125
Name: label, dtype: float64

1. Since, the target feature i.e `label` has only two unique values i.e `[0,1]`, hence, it's a **`Binary Classification`** problem.

## Downsampling the dataset
A positive label indicates that the center 32x32 px region of a patch contains at least one px of tumor tissue. There are **130,908 normal cases (0)** and **89,117 abnormal (or cancerous) tissue images (1)**. This is a huge dataset that will require a lot of time and compute resources to train on a full dataset, thus we will downsample the 220,000 images in the train folder to 10,000 images and then split them into training and testing datasets:

In [8]:
np.random.seed(42)
train_imgs_orig = os.listdir("/kaggle/input/histopathologic-cancer-detection/train")
selected_image_list = []

for img in np.random.choice(train_imgs_orig, 10000):
    selected_image_list.append(img)
    
len(selected_image_list)

10000

## Train Test split

In [9]:
np.random.seed(42)
np.random.shuffle(selected_image_list)

train_idx = selected_image_list[:8000]
test_idx = selected_image_list[8000:]

print(f"Number of images in the downsampled training dataset: {len(train_idx)}")
print(f"Number of images in the downsampled testing dataset: {len(test_idx)}")

Number of images in the downsampled training dataset: 8000
Number of images in the downsampled testing dataset: 2000


In [10]:
### downsampled_train_dataset
os.mkdir('/kaggle/working/downsampled_train_dataset')

for file_name in train_idx:
    source = os.path.join('/kaggle/input/histopathologic-cancer-detection/train',file_name)
    destination = os.path.join('/kaggle/working/downsampled_train_dataset/', file_name)
    shutil.copyfile(source, destination)

### downsampled_test_dataset
os.mkdir('/kaggle/working/downsampled_test_dataset')

for file_name in test_idx:
    # we used the train datset instead. of test because we are using the train datset itself for both training & testing
    source = os.path.join('/kaggle/input/histopathologic-cancer-detection/train',file_name)
    destination = os.path.join('/kaggle/working/downsampled_test_dataset/', file_name)
    shutil.copyfile(source, destination)
    
print("Copying of data from Source to Destination is completed!!!")

Copying of data from Source to Destination is completed!!!


In the preceding code snippet, we are first creating two folders called **`downsampled_train_dataset * downsampled_test_dataset`** on the following path:
```python
os.mkdir('/kaggle/working/downsampled_train_dataset')
os.mkdir('/kaggle/working/downsampled_test_dataset')
```
Then, we are looping over `train_idx` to get image names for the training data and copying all the files from input storage in the machine allotted to us at the time we created a notebook to working storage on our Kaggle using the **shutils** Python module.

## Label extraction

The labels for the images that were selected in the downsampled data will be extracted in a list that will be used for training and evaluating the image classification model, as shown here:

In [11]:
selected_image_labels = pd.DataFrame()

id_list = []
label_list = []

for img in selected_image_list:
    label_tuple = data_df.loc[data_df['id'] == img.split('.')[0]]
    id_list.append(label_tuple['id'].values[0])
    label_list.append(label_tuple['label'].values[0])  
    
print(id_list[:5])
print(label_list[:5])

['56a2b15a4da049186b939e872086a08c0b6406d6', 'bb64f942aafd122e8247a1bc057735f35877e6a6', '346e45de7df976a890dcd46115e90befd753bf55', 'fc92ba278f306c1d34f7f783ee5c0740a6541b5d', 'cff1f67703a6dcbc55983e1ecfa0f60d6294c1ef']
[0, 1, 0, 0, 0]


## Loading the dataset
**PyTorch Lightning** `expects data to be in folders with the classes.` So, we cannot use the DataLoader module directly when all train/test images are in one folder without subfolders. Therefore, we will write our custom class for loading the data, as follows:

In [12]:
class LoadCancerDataset(Dataset):
    
    def __init__(self, data_folder, transform = T.Compose([T.CenterCrop(32),T.ToTensor()]), dict_labels={}):
        self.data_folder = data_folder
        self.list_image_files = [s for s in os.listdir(data_folder)]
        self.transform = transform
        self.dict_labels = dict_labels
        self.labels = [dict_labels[i.split('.')[0]] for i in self.list_image_files]
        
    def __len__(self):
        return len(self.list_image_files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.data_folder, self.list_image_files[idx])
        image = Image.open(img_name)
        image = self.transform(image)[0]
        img_name_short = self.list_image_files[idx].split('.')
        label = self.dict_labels[img_name_short]
        return image, label

In the preceding code block, we have defined a custom data loader.

The custom class defined earlier inherits from the `torch.utils.data.Dataset` module. The **LoadCancerDataset** custom class is initialized in the **__init__** method and accepts three arguments: **the `path to the data folder`, the `transformer` with a default value of cropping the image to size 32 and transforming it to a tensor, and a `dictionary` with the labels and IDs of the dataset.**

The **LoadCancerDataset** class reads all the images in the folder and **extracts** the image name from the filename, which is also the `ID` for the images. This image name is then matched with the label in the dictionary with the labels and IDs.

The **LoadCancerDataset** class returns the images and their labels, which can then be used in the DataLoader module of the `torch.utils.data` library as it can now read the images with their corresponding label.

## Augmenting the dataset
Now that we have loaded the data, we will start the process of data preprocessing by augmenting the images, as follows:

In [13]:
data_T_train = T.Compose([ T.CenterCrop(32), T.RandomHorizontalFlip(), T.RandomVerticalFlip(), T.ToTensor(), ])

data_T_test = T.Compose([ T.CenterCrop(32), T.ToTensor(), ])

In the preceding code block,

We have used transformations to crop the image to 32x32 by using **Torchvision's** built-in libraries. We then also **augmented the data** by flipping it horizontally and vertically, **thereby creating two additional copies from the original image.**

Now, we will call our **LoadCancerDataset** custom class with the path to the data folder, transformer, and the image label dictionary to convert it to the format accepted by the `torch.utils.data.DataLoader` module.

In [14]:
train_set = LoadCancerDataset(data_folder='/kaggle/working/downsampled_train_dataset',transform=data_T_train,dict_labels={})
test_set = LoadCancerDataset(data_folder='/kaggle/working/downsampled_test_dataset',transform=data_T_test,dict_labels={})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 train_set = LoadCancerDataset(data_folder='/kaggle/working/downsampled_train_dataset',tr     │
│   2 test_set = LoadCancerDataset(data_folder='/kaggle/working/downsampled_test_dataset',tran     │
│   3                                                                                              │
│                                                                                                  │
│ in __init__:8                                                                                    │
│                                                                                                  │
│    5 │   │   self.list_image_files = [s for s in os.listdir(data_folder)]                        │
│    6 │   │   self.transform = transform                                                          │
│    7 │   │   self.dict_labels = dict_labels                                                      │
│ ❱  8 │   │   self.labels = [dict_labels[i.split('.')[0]] for i in self.list_image_files]         │
│    9 │                                                                                           │
│   10 │   def __len__(self):                                                                      │
│   11 │   │   return len(self.list_image_files)                                                   │
│                                                                                                  │
│ in <listcomp>:8                                                                                  │
│                                                                                                  │
│    5 │   │   self.list_image_files = [s for s in os.listdir(data_folder)]                        │
│    6 │   │   self.transform = transform                                                          │
│    7 │   │   self.dict_labels = dict_labels                                                      │
│ ❱  8 │   │   self.labels = [dict_labels[i.split('.')[0]] for i in self.list_image_files]         │
│    9 │                                                                                           │
│   10 │   def __len__(self):                                                                      │
│   11 │   │   return len(self.list_image_files)                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: '7cd369c04a37c9da20bbfe1bcba2cfad754fc100'

This will be repeated for the test data as well, and then, in the final step, we will create train_dataloader and test_dataloader data loaders using the output from our LoadCancerDataset custom class by leveraging the DataLoader module from the torch.utils.data library. The code to do so is illustrated in the following snippet:


In [ ]:
batch_size = 256

train_dataloader = DataLoader(train_set, batch_size, num_workers=2, pin_memory=True, shuffle=True)

test_dataloader = DataLoader(test_set, batch_size, num_workers=2, pin_memory=True)

In the preceding code snippet, we have the following:
1. We started with the original data without the subfolders as expected by the PyTorch Lightning module. The data was downsampled and saved on Google Drive's persistent storage.
2. Using the LoadCancerDataset custom class, we created two datasets train_ set and test_set, by reading images and their labels.
3. In the process of creating datasets, we also used the Torchvision transform module to crop the images to the center, that is, converting images to the square of 32 x 32 px and also converting images to tensors.
4. In the final step, the two train_set and test_set datasets that were created are used to create two train_dataloader and test_dataloader data loaders for them.

At this point, we are ready with our train_dataloader data loader with around 8,000 images, and test_dataloader with around 2,000 images. All the images are of size 32 x 32, converted to tensor form, and served in batches of 256 images. We will use the train data loader to train our model and the test data loader to measure our model's accuracy.